# 1. 데이터 읽기

colab에서 작성 및 실행하고, 구글드라이브에서 데이터를 가져왔다.

In [5]:
!unzip /content/drive/MyDrive/weather_2019_2021.zip -d /content/weather
!unzip /content/drive/MyDrive/elicer_col_name.zip -d /content/elice_배달데이터

Archive:  /content/drive/MyDrive/weather_2019_2021.zip
  inflating: /content/weather/dust_2019_0801_2020_0801.csv  
  inflating: /content/weather/dust_2020_0802_2021_0831.csv  
  inflating: /content/weather/weather_2019_0801_2020_0801.csv  
  inflating: /content/weather/weather_2020_0802_2021_0831.csv  


In [6]:
import numpy as np
import pandas as pd 
import missingno as msno

#iplot사용
#interactive한 그래프를 그릴 수 있다.
import plotly.graph_objs as go 
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [7]:
#stackoverflow
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

## 1.1 데이터 불러오기

In [22]:
weather1=pd.read_csv("/content/weather/weather_2019_0801_2020_0801.csv", encoding = 'cp949', parse_dates=['일시'])
weather2=pd.read_csv("/content/weather/weather_2020_0802_2021_0831.csv", encoding = 'cp949',parse_dates=['일시'])
dust1=pd.read_csv("/content/weather/dust_2019_0801_2020_0801.csv", encoding = 'cp949',parse_dates=['일시'])
dust2=pd.read_csv("/content/weather/dust_2020_0802_2021_0831.csv", encoding = 'cp949',parse_dates=['일시'])

In [11]:
weather1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836075 entries, 0 to 836074
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   지점명      836075 non-null  object        
 1   일시       836075 non-null  datetime64[ns]
 2   기온(°C)   836054 non-null  float64       
 3   강수량(mm)  87381 non-null   float64       
 4   적설(cm)   5153 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 31.9+ MB


In [25]:
print(weather1.tail())
weather2

       지점명                  일시  기온(°C)  강수량(mm)  적설(cm)
836070  남해 2020-08-01 19:00:00    26.9      NaN     NaN
836071  남해 2020-08-01 20:00:00    26.7      NaN     NaN
836072  남해 2020-08-01 21:00:00    26.7      NaN     NaN
836073  남해 2020-08-01 22:00:00    26.6      NaN     NaN
836074  남해 2020-08-01 23:00:00    26.1      NaN     NaN


,지점명,일시,기온(°C),강수량(mm),적설(cm)
0,속초,2020-08-02 00:00:00,23.2,NaN,NaN
1,속초,2020-08-02 01:00:00,22.9,NaN,NaN
2,속초,2020-08-02 02:00:00,23.0,NaN,NaN
3,속초,2020-08-02 03:00:00,23.2,0.8,NaN
4,속초,2020-08-02 04:00:00,23.1,NaN,NaN
...,...,...,...,...,...
899728,남해,2021-08-31 19:00:00,26.0,NaN,NaN
899729,남해,2021-08-31 20:00:00,25.7,NaN,NaN
899730,남해,2021-08-31 21:00:00,25.3,NaN,NaN
899731,남해,2021-08-31 22:00:00,25.7,NaN,NaN


In [42]:
print(dust1.tail())
dust2

        지점명                  일시  1시간평균 미세먼지농도(㎍/㎥)  Unnamed: 3
231189  연평도 2020-08-01 19:00:00                 32         NaN
231190  연평도 2020-08-01 20:00:00                 33         NaN
231191  연평도 2020-08-01 21:00:00                 23         NaN
231192  연평도 2020-08-01 22:00:00                 17         NaN
231193  연평도 2020-08-01 23:00:00                 16         NaN


,지점명,일시,1시간평균 미세먼지농도(㎍/㎥)
0,속초,2020-08-02 00:00:00,19
1,속초,2020-08-02 01:00:00,31
2,속초,2020-08-02 02:00:00,22
3,속초,2020-08-02 03:00:00,25
4,속초,2020-08-02 04:00:00,16
...,...,...,...
247793,연평도,2021-08-31 19:00:00,8
247794,연평도,2021-08-31 20:00:00,8
247795,연평도,2021-08-31 21:00:00,17
247796,연평도,2021-08-31 22:00:00,5


##1.2 데이터 프레임 하나로 합치기

In [52]:
weather = pd.concat([weather1, weather2], ignore_index=True)
dust = pd.concat([dust1, dust2], ignore_index=True)

In [58]:
#csv열었을때는 안보였는데 dataframe으로 만드니까 'Unnamed: 3'라는 열이 생겼다. 따라서 제거해준다.
#정제카테고리에서 해주어야하는 일인가?
dust = dust.drop('Unnamed: 3', axis=1)
dust

,지점명,일시,1시간평균 미세먼지농도(㎍/㎥)
0,속초,2019-08-01 00:00:00,12
1,속초,2019-08-01 01:00:00,11
2,속초,2019-08-01 02:00:00,12
3,속초,2019-08-01 03:00:00,4
4,속초,2019-08-01 05:00:00,6
...,...,...,...
478987,연평도,2021-08-31 19:00:00,8
478988,연평도,2021-08-31 20:00:00,8
478989,연평도,2021-08-31 21:00:00,17
478990,연평도,2021-08-31 22:00:00,5


In [14]:
df_2019=pd.read_csv("/content/elice_배달데이터/elicer_2019_col_name.csv", parse_dates=['날짜'])
df_2020_1=pd.read_csv("/content/elice_배달데이터/elicer_2020_1_col_name.csv", parse_dates=['날짜'])
df_2020_2=pd.read_csv("/content/elice_배달데이터/elicer_2020_2_col_name.csv", parse_dates=['날짜'])
df_2021=pd.read_csv("/content/elice_배달데이터/elicer_2021_col_name.csv", parse_dates=['날짜'])

In [15]:
df = pd.concat([df_2019, df_2020_1, df_2020_2, df_2021], ignore_index=True)
df

,날짜,시간대,광역시도,시군구,읍면동,배달건수
0,2019-08-01,0,강원도,속초시,교동,1
1,2019-08-01,0,경기도,고양시 일산동구,마두동,4
2,2019-08-01,0,경기도,고양시 일산동구,백석동,28
3,2019-08-01,0,경기도,고양시 일산동구,식사동,3
4,2019-08-01,0,경기도,고양시 일산동구,장항동,4
...,...,...,...,...,...,...
2382976,2021-08-31,23,충청남도,천안시 서북구,쌍용동,6
2382977,2021-08-31,23,충청북도,제천시,신백동,6
2382978,2021-08-31,23,충청북도,제천시,신월동,2
2382979,2021-08-31,23,충청북도,제천시,천남동,1


# 2. 데이터 정제

**정제 해야할것** 
1. 일시를 날짜와 시간으로 나눈다. 엘리스 배달 데이터셋과 같은 형식으로 저장한다.
2. **NaN**을 모두 0으로 => 강수량과 적설은 가능한데, 기온은 0도가 존재하기 때문에 0은 불가능!
3. 지점명을 통일하는 것이 가장 큰 문제.
배달데이터는 광역시도와 시군구가 따로 저장되어있는데 날씨데이터에는 섞여서 저장되어있음에도 불구하고 숫자도 더 적다...심지어 미세먼지 데이터는 지역이 29개밖에 안되서 어떤지역이 어떤지점의 데이터를 가져다 쓰는지도 알아야한다.

## 2.1 결측치 확인

In [31]:
weather.isna().sum()

지점명              0
일시               0
기온(°C)         121
강수량(mm)    1556729
적설(cm)     1705438
dtype: int64

In [40]:
dust.isna().sum()

지점명                  0
일시                   0
1시간평균 미세먼지농도(㎍/㎥)    0
dtype: int64

## 2.2 중복값 확인

In [41]:
print("weather-----\n",weather[weather.duplicated(keep=False)])
print("\ndust-----\n",dust[dust.duplicated(keep=False)])

weather-----
 Empty DataFrame
Columns: [지점명, 일시, 기온(°C), 강수량(mm), 적설(cm)]
Index: []

dust-----
 Empty DataFrame
Columns: [지점명, 일시, 1시간평균 미세먼지농도(㎍/㎥)]
Index: []


## 2.3 배달데이터와 날짜, 시간대 형식 맞추기

In [56]:
weather.head()

,지점명,일시,기온(°C),강수량(mm),적설(cm)
0,속초,2019-08-01 00:00:00,30.3,NaN,NaN
1,속초,2019-08-01 01:00:00,30.1,NaN,NaN
2,속초,2019-08-01 02:00:00,29.1,0.3,NaN
3,속초,2019-08-01 03:00:00,29.5,NaN,NaN
4,속초,2019-08-01 04:00:00,29.2,NaN,NaN


In [59]:
dust.head()

,지점명,일시,1시간평균 미세먼지농도(㎍/㎥)
0,속초,2019-08-01 00:00:00,12
1,속초,2019-08-01 01:00:00,11
2,속초,2019-08-01 02:00:00,12
3,속초,2019-08-01 03:00:00,4
4,속초,2019-08-01 05:00:00,6


In [60]:
df.head()

,날짜,시간대,광역시도,시군구,읍면동,배달건수
0,2019-08-01,0,강원도,속초시,교동,1
1,2019-08-01,0,경기도,고양시 일산동구,마두동,4
2,2019-08-01,0,경기도,고양시 일산동구,백석동,28
3,2019-08-01,0,경기도,고양시 일산동구,식사동,3
4,2019-08-01,0,경기도,고양시 일산동구,장항동,4


In [61]:
#datetime형식에서 일자와 시간만 각각 분리해서 날짜와 시간대로 새로 저장
weather['날짜'] = weather['일시'].dt.date
weather['시간대'] = weather['일시'].dt.hour
dust['날짜'] = dust['일시'].dt.date
dust['시간대'] = dust['일시'].dt.hour

#기존 일시 항목은 삭제
weather = weather.drop('일시', axis=1)
dust = dust.drop('일시', axis=1)

#column순서 바꾸기
weather = weather[['날짜','시간대', '지점명','기온(°C)'	,'강수량(mm)','적설(cm)']]
dust = dust[['날짜','시간대','지점명','1시간평균 미세먼지농도(㎍/㎥)']]

##csv로 저장(db에 넣기전 위와 같이 정제 or csv파일을 새롭게 저장한뒤 그 파일을 사용해서 저장)
# weather.to_csv("/content/drive/MyDrive/weather_2019_2021_new.csv", index=False, encoding="utf-8-sig")
# dust.to_csv("/content/dirve/MyDrive/dust_2019_2021_new.csv", index=False, encoding="utf-8-sig")

In [62]:
weather

,날짜,시간대,지점명,기온(°C),강수량(mm),적설(cm)
0,2019-08-01,0,속초,30.3,NaN,NaN
1,2019-08-01,1,속초,30.1,NaN,NaN
2,2019-08-01,2,속초,29.1,0.3,NaN
3,2019-08-01,3,속초,29.5,NaN,NaN
4,2019-08-01,4,속초,29.2,NaN,NaN
...,...,...,...,...,...,...
1735803,2021-08-31,19,남해,26.0,NaN,NaN
1735804,2021-08-31,20,남해,25.7,NaN,NaN
1735805,2021-08-31,21,남해,25.3,NaN,NaN
1735806,2021-08-31,22,남해,25.7,NaN,NaN


In [63]:
dust

,날짜,시간대,지점명,1시간평균 미세먼지농도(㎍/㎥)
0,2019-08-01,0,속초,12
1,2019-08-01,1,속초,11
2,2019-08-01,2,속초,12
3,2019-08-01,3,속초,4
4,2019-08-01,5,속초,6
...,...,...,...,...
478987,2021-08-31,19,연평도,8
478988,2021-08-31,20,연평도,8
478989,2021-08-31,21,연평도,17
478990,2021-08-31,22,연평도,5


## 2.4 (HARD) 배달데이터와 장소 맞추기 

In [64]:
print(df['광역시도'].nunique())
df['광역시도'].unique()

17


array(['강원도', '경기도', '경상남도', '대전광역시', '서울특별시', '전라남도', '전라북도', '제주특별자치도',
       '충청남도', '충청북도', '경상북도', '대구광역시', '부산광역시', '광주광역시', '인천광역시',
       '울산광역시', '세종특별자치시'], dtype=object)

In [18]:
print(df['시군구'].nunique())
df['시군구'].unique()

141


array(['속초시', '고양시 일산동구', '고양시 일산서구', '광명시', '시흥시', '의정부시', '포천시', '김해시',
       '진주시', '동구', '구로구', '금천구', '마포구', '양천구', '영등포구', '은평구', '순천시',
       '전주시 완산구', '서귀포시', '서산시', '제천시', '안동시', '김포시', '남양주시', '안산시 상록구',
       '창원시 마산회원구', '서구', '강서구', '도봉구', '성북구', '부천시', '양주시', '용인시 처인구',
       '파주시', '화성시', '창원시 마산합포구', '함양군', '달서구', '달성군', '사하구', '강북구',
       '보령시', '원주시', '연천군', '평택시', '창녕군', '해운대구', '노원구', '동작구', '부평구',
       '수원시 권선구', '천안시 동남구', '청주시 청원구', '북구', '나주시', '사천시', '여수시', '서대문구',
       '제주시', '남동구', '군산시', '삼척시', '구미시', '강릉시', '관악구', '수영구', '수성구',
       '수원시 장안구', '홍성군', '포항시 북구', '고양시 덕양구', '양평군', '경주시', '성남시 수정구',
       '여주시', '\\N', '익산시', '울주군', '천안시 서북구', '동두천시', '광양시', '충주시', '송파구',
       '안양시 만안구', '창원시 의창구', '강남구', '영주시', '중구', '칠곡군', '용인시 수지구', '유성구',
       '영천시', '경산시', '서초구', '양산시', '기장군', '구리시', '창원시 성산구', '연수구', '연제구',
       '용산구', '용인시 기흥구', '대덕구', '동래구', '청주시 흥덕구', '안양시 동안구', '동대문구',
       '광주시', '거제시', '수원시 팔달구', '강동구', '음성군', '문경시', '김천시', '오산

In [34]:
print(weather['지점명'].nunique())
weather['지점명'].unique()

96


array(['속초', '북춘천', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉',
       '동해', '서울', '인천', '원주', '울릉도', '수원', '영월', '충주', '서산', '울진', '청주',
       '대전', '추풍령', '안동', '상주', '포항', '군산', '대구', '전주', '울산', '창원', '광주',
       '부산', '통영', '목포', '여수', '흑산도', '완도', '고창', '순천', '진도(첨찰산)', '홍성',
       '제주', '고산', '성산', '서귀포', '진주', '강화', '양평', '이천', '인제', '홍천', '태백',
       '정선군', '제천', '보은', '천안', '보령', '부여', '금산', '세종', '부안', '임실', '정읍',
       '남원', '장수', '고창군', '영광군', '김해시', '순창군', '북창원', '양산시', '보성군', '강진군',
       '장흥', '해남', '고흥', '의령군', '함양군', '광양시', '진도군', '봉화', '영주', '문경',
       '청송군', '영덕', '의성', '구미', '영천', '경주시', '거창', '합천', '밀양', '산청', '거제',
       '남해'], dtype=object)

In [35]:
print(dust['지점명'].nunique())
dust['지점명'].unique()

29


array(['속초', '북춘천', '광덕산', '대관령', '백령도', '서울', '울릉도독도(감)', '관악산', '수원',
       '영월', '울진', '안면도(감)', '추풍령', '안동', '군산', '대구', '전주', '울산', '광주',
       '구덕산', '흑산도', '고산', '진주', '강화', '북격렬비도', '천안', '진도군', '문경', '연평도'],
      dtype=object)